<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/t5_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers==4.6.0

In [ ]:
!pip install tensorflow-addons==0.13.0

In [ ]:
!pip install --upgrade sentencepiece==0.1.91

In [4]:
import pandas as pd
from sklearn.utils import shuffle

In [5]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

In [35]:
from transformers import TFT5ForConditionalGeneration, T5Tokenizer, BatchEncoding

In [17]:
from tqdm.auto import tqdm, trange
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv

--2022-02-07 09:26:47--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200691 (196K) [text/plain]
Saving to: ‘dev.tsv’

dev.tsv             100%[===================>] 195.99K  --.-KB/s    in 0.02s   

2022-02-07 09:26:47 (8.27 MB/s) - ‘dev.tsv’ saved [200691/200691]



In [49]:
base_model_name = 'sberbank-ai/ruT5-base'
model_name = 'SkolkovoInstitute/ruT5-base-detox'

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [50]:
ds = pd.read_csv('/content/dev.tsv', sep='\t')
toxic_inputs = ds['toxic_comment'].tolist()

In [51]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids']
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs, 
        num_return_sequences=5, 
        do_sample=False, 
        temperature=temperature, 
        repetition_penalty=3.0, 
        max_length=max_length,
        bad_words_ids=[[2]],
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [95]:
print(paraphrase(['Ублюдок, мать твою, а ну иди сюда'], model, temperature=5.0, beams=5))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


['А ну иди сюда', 'а ну иди сюда', 'Нехороший человек, а ну иди сюда', 'Нехороший человек, а ну иди сюда.', 'Нехороший, а ну иди сюда']


In [107]:
para_results = []
batch_size = 8

for i in tqdm(range(0, len(toxic_inputs), batch_size)):
    batch = [sentence for sentence in toxic_inputs[i:i + batch_size]]
    para_results.extend(paraphrase(batch, model, temperature=0.0, beams=5))

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [108]:
print(len(para_results))

4000


In [109]:
print(para_results[:10])

['Враньё! температуры горения хватит чтобы ее расплавить', 'враньё! температуры горения хватит чтобы ее расплавить', 'Враньё! температуры горения хватит, чтобы ее расплавить', 'Враньё! температуры горения хватит чтобы её расплавить', 'враньё! температуры горения хватит чтобы её расплавить', 'а ты там был.ты вообще служил.', 'А ты где был? Ты вообще служил', 'А ты где был? Ты вообще служил.', 'а ты там был.ты вообще служил', 'А ты там был. Ты вообще служил']


In [111]:
with open('/content/t5_base_dev_large.txt', 'w') as file:
  file.writelines(["%s\n" % item  for item in para_results])